In [566]:
import pandas as pd
import numpy as np
import regex as re
from io import StringIO
import string
from datetime import datetime


# NEW

In [567]:
doc = "output-4-pandas.csv"
pd_csv = pd.read_csv(doc)
pd_csv = pd_csv.iloc[:, 1:]
# Resetting index
# Find out where portfolio company is
for row in range(5):
    #print(pd_csv.iloc[row])
    if 'Company' in str(pd_csv.iloc[row]):
        print("Company Found")
        portfolio_index = row
        break

# Checking if all the column are in the row with portfolio and adding them if they are not.
for j in range(pd_csv.shape[1]):
    if (pd_csv.iat[portfolio_index - 1, j] == 'Principal'):
        pd_csv.iat[portfolio_index, j] = 'Principal'
    elif (pd_csv.iat[portfolio_index - 1, j] == 'Cost'):
        pd_csv.iat[portfolio_index, j] = 'Cost'
    elif (pd_csv.iat[portfolio_index - 1, j] == 'Fair'):
        pd_csv.iat[portfolio_index, j] = 'Fair Value'
    elif ('Principal' in str(pd_csv.iat[portfolio_index - 1, j])):
        pd_csv.iat[portfolio_index, j] = 'Principal'

print("Portofolio", portfolio_index)
# Removing rows before portfolio index
if (pd_csv.iloc[portfolio_index].str.contains('Company').any() == True):
    print("Inside")
    pd_csv = pd_csv.iloc[portfolio_index:]


Company Found
Portofolio 1
Inside


In [568]:
if(pd_csv['0'].isnull().sum()/len(pd_csv)==1):
    pd_csv = pd_csv.drop(pd_csv.columns[0], axis=1)

In [569]:
pd_csv[:30]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
1,Portfolio Company,NaN,Industry,NaN,Type of Investment (1)(2),NaN,Principal Amount,NaN,Cost,NaN,Fair Value (3),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Non–Control / Non–Affiliate Investments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Agilex Flavors & Fragrances, Inc. (3%)*",NaN,Custom Fragrance Producers,NaN,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",NaN,NaN,12506771,NaN,NaN,NaN,12256771.0,NaN,NaN,NaN,12256771.0,NaN,NaN
4,"Common Units (1,250 units)",NaN,NaN,NaN,1250000,NaN,NaN,1250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,12506771,NaN,NaN,13506771,NaN,NaN,13506771.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Ambient Air Corporation (“AA”) and Peaden-Hobb...,NaN,Specialty Trade Contractors,NaN,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",NaN,4047120,NaN,NaN,4038351.0,NaN,NaN,4038351.0,NaN,NaN,NaN,NaN,NaN
7,"Subordinated Note-PHM (12% Cash, Due 09/12)",NaN,12857,NaN,NaN,12857.0,NaN,NaN,12857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Common Stock-PHM (128,571 shares)",NaN,NaN,NaN,128571,NaN,NaN,128571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Common Stock Warrants-AA (455 shares),NaN,NaN,NaN,142361,NaN,NaN,242000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,4059977,NaN,NaN,4322140,NaN,NaN,4421779.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [570]:
pd_csv = pd_csv.reset_index(drop=True)
second_header = list()
#Identifying where the second header is
for index, row in pd_csv.iterrows():
    #print(row[0],row[2])
    if index<5:
        continue
    if 'portfolio' in str(row[1]).lower():
        second_header.append(index)
    elif 'industry' in str(row[2]).lower():
        second_header.append(index)
    elif 'portfolio' in str(row[2]).lower():
        second_header.append(index)
#Dropping the second header
try:
    print("Second Header",second_header)
    pd_csv.drop(second_header, inplace=True)
except:
    print("No second header")

Second Header [50, 98, 147, 193, 239]


In [571]:
pd_csv.loc[pd_csv['0'].isna()] = pd_csv.loc[pd_csv['0'].isna()].shift(-1, axis=1)  #Changes

In [572]:
#Identifying levels in the dataframe
listOfLevels = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if (pd.notnull(pd_csv.iat[i, j])):
            #print(pd_csv.iat[i, j])
            if ((('investments' in str(pd_csv.iat[i, j])) or ('Investments' in str(pd_csv.iat[i, j]))) and
                    ('Subtotal' not in pd_csv.iat[i, j]) and ('Total' not in pd_csv.iat[i, j]) and
                    ('inc' not in str(pd_csv.iat[i, j].lower())) and ('Fund' not in pd_csv.iat[i, j]) and 
                    ('LLC' not in pd_csv.iat[i, j]) and ('L.P' not in pd_csv.iat[i, j])  and 
                    ('CMS' not in pd_csv.iat[i, j]) and ('holdings' not in str(pd_csv.iat[i, j].lower()))):
                listOfLevels.append(i)

In [573]:
pd_csv[140:200]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
142,"Stella Environmental Services, LLC (1%)*",NaN,Waste Transfer Stations,NaN,"Subordinated Notes (12% Cash, 3% PIK, Due 2/17)",NaN,5916326,NaN,NaN,5788004.0,NaN,NaN,5788004.0,NaN,NaN,NaN,NaN,NaN
143,Common Stock Purchase Warrants,NaN,NaN,NaN,20000,NaN,NaN,259000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,NaN,NaN,5916326.0,NaN,NaN,5808004,NaN,NaN,6047004.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,"Syrgis Holdings, Inc. (0%)*",NaN,Specialty Chemical Manufacturer,NaN,"Class C Units (2,114 units)",NaN,NaN,NaN,111037,NaN,NaN,201281.0,NaN,NaN,NaN,NaN,NaN,NaN
146,NaN,NaN,NaN,111037,NaN,NaN,201281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,The Krystal Company (4%)*,NaN,Quick Serve Restaurants,NaN,"Subordinated Note (12% Cash, 3% PIK, Due 6/17)",NaN,NaN,12514671,NaN,NaN,NaN,12296595.0,NaN,NaN,NaN,12296595.0,NaN,NaN
149,"Class A Units of Limited Partnership (2,000 un...",NaN,NaN,NaN,2000000,NaN,NaN,3051000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,NaN,NaN,NaN,12514671,NaN,NaN,14296595,NaN,NaN,15347595.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,"TMR Automotive Service Supply, LLC (1%)*",NaN,Automotive Supplies,NaN,"Subordinated Note (12% Cash, 1% PIK, Due 03/16)",NaN,4750000,NaN,NaN,4539123.0,NaN,NaN,4539123.0,NaN,NaN,NaN,NaN,NaN
152,"Unit Purchase Warrant (316,858 units)",NaN,NaN,NaN,195000,NaN,NaN,384000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [574]:
#Pattern
pattern = r"\d{1,3}(,\d{3})*\s+(shares|units|cash|warrant(\s*\(\d+\%\))?)|\d+\%\s+cash|\broyalty\s+rights\b|warrant|units|(preferred interest)"


# Find the index of the "Type of Investment" column
type_col_idx = pd_csv.iloc[:2].apply(lambda x: x.astype(str).str.contains("Type of Investment", case=False)).any().argmax()

# Shift rows to move matched value in the first column to the column under "Type of Investment"
for i, row in pd_csv.iterrows():
    if i == 0:
        continue  # Skip the first row
    if bool(re.search(pattern, str(row[0]).lower())):
        # Shift the row to move the matched value in the first column to the column under "Type of Investment"
        pd_csv.loc[i,:] = pd_csv.loc[i,:].shift(3)
    if bool(re.search(pattern, str(row[1]).lower())):
        # Shift the row to move the matched value in the first column to the column under "Type of Investment"
        pd_csv.loc[i,:] = pd_csv.loc[i,:].shift(2)

In [575]:
pd_csv[100:160]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
102,"Media Temple, Inc. (4%)*",NaN,Web Hosting Services,NaN,"Subordinated Note (12% Cash, 3% PIK, Due 04/15)",NaN,8800000,NaN,NaN,8696378.0,NaN,NaN,8696378.0,NaN,NaN,NaN,NaN,NaN
103,None,None,None,"Convertible Note (8% Cash, 6% PIK, Due 04/15)",NaN,3200000,NaN,NaN,2896501.0,NaN,NaN,6377000,NaN,NaN,NaN,NaN,NaN,NaN
104,None,None,None,"Common Stock Purchase Warrant (28,000 shares)",NaN,NaN,NaN,536000,NaN,NaN,2790000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,NaN,12000000,NaN,NaN,12128879.0,NaN,NaN,17863378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,Micross Solutions LLC (3%)*,NaN,Provider Semiconductor Products and Services,NaN,"Subordinated Note (12% Cash, 2% PIK, Due 06/18)",NaN,10615311,NaN,NaN,10456311.0,NaN,NaN,10456311.0,NaN,NaN,NaN,NaN,NaN
107,None,None,None,"Class A-2 Common Units (1,500,000 units)",NaN,NaN,NaN,1500000,NaN,NaN,1500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,10615311,NaN,NaN,11956311.0,NaN,NaN,11956311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,"Minco Technology Labs, LLC (1%)*",NaN,Semiconductor Distribution,NaN,"Subordinated Note (6.5% Cash, 3.5% PIK, Due 05...",NaN,5537286,NaN,NaN,5453091.0,NaN,NaN,3619000.0,NaN,NaN,NaN,NaN,NaN
110,None,None,None,"Class A Units (5,000 Holdco units)",NaN,NaN,NaN,500000,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,None,None,None,"Class A Units (3,907 OpCo. units)",NaN,NaN,NaN,3907,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [576]:
remove = string.punctuation
remove = remove.replace("-", "") # don't remove hyphens
pattern = r"[{}]".format(remove) # create the pattern
pd_csv = pd_csv.reset_index(drop=True)

# Adding the Levels to an array
newValues = []
#newValues.append("Levels")

for i in range(len(listOfLevels)):
    beginIndex = listOfLevels[i]
    if (i == len(listOfLevels) - 1):
        endIndex = len(pd_csv)
    else:
        endIndex = listOfLevels[i + 1]
    value = pd_csv.iat[beginIndex, 0]
    if(value == 'Portfolio Company'):
        value = 'Levels'
    try:
        value = ''.join([i for i in value if not i.isdigit()])
        value = re.sub("[^\P{P}-]+", "", value)
    except:
        value = pd_csv.iat[beginIndex, 0]
    print("Begin",beginIndex)
    print("End",endIndex)
    for x in range(beginIndex, endIndex):
        newValues.append(value)

#print(newValues)
if(len(newValues)<(len(pd_csv))):
    newValues.insert(0, 'Levels')
# Adding the array to the df
pd_csv.insert(0, 'Levels', newValues)
 

Begin 1
End 189
Begin 189
End 258
Begin 258
End 279


In [577]:
len(newValues)

279

In [578]:
pd_csv.head(30)

,Levels,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,Levels,Portfolio Company,NaN,Industry,NaN,Type of Investment (1)(2),NaN,Principal Amount,NaN,Cost,NaN,Fair Value (3),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NonControl NonAffiliate Investments,Non–Control / Non–Affiliate Investments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",NaN,Custom Fragrance Producers,NaN,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",NaN,NaN,12506771,NaN,NaN,NaN,12256771.0,NaN,NaN,NaN,12256771.0,NaN,NaN
3,NonControl NonAffiliate Investments,None,None,None,"Common Units (1,250 units)",NaN,NaN,NaN,1250000,NaN,NaN,1250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NonControl NonAffiliate Investments,NaN,NaN,12506771,NaN,NaN,13506771,NaN,NaN,13506771.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,NaN,Specialty Trade Contractors,NaN,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",NaN,4047120,NaN,NaN,4038351.0,NaN,NaN,4038351.0,NaN,NaN,NaN,NaN,NaN
6,NonControl NonAffiliate Investments,None,None,None,"Subordinated Note-PHM (12% Cash, Due 09/12)",NaN,12857,NaN,NaN,12857.0,NaN,NaN,12857,NaN,NaN,NaN,NaN,NaN,NaN
7,NonControl NonAffiliate Investments,None,None,None,"Common Stock-PHM (128,571 shares)",NaN,NaN,NaN,128571,NaN,NaN,128571,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NonControl NonAffiliate Investments,None,None,None,Common Stock Warrants-AA (455 shares),NaN,NaN,NaN,142361,NaN,NaN,242000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NonControl NonAffiliate Investments,NaN,NaN,4059977,NaN,NaN,4322140,NaN,NaN,4421779.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [579]:
# Finding Subtotals
pd_csv.reset_index()
subtotal_list = list()
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if (('Subtotal' in str(pd_csv.iat[i, j]))):
            # print(pd_csv.iloc[i,:])
            subtotal_list.append(i)

print("Subtotal", subtotal_list)
# Dropping the subtotal rows
pd_csv.drop(pd_csv.index[subtotal_list], inplace=True)

# Finding Totals
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if ('Total' in str(pd_csv.iat[i, j])):
            total_list.append(i)
            
print("Total list",total_list)
pd_csv.drop(pd_csv.index[total_list], inplace=True)
# Dropping everything after the total rows
try:
    pd_csv = pd_csv.iloc[:total_list[-1]]
except:
    print("Total not found")
    
for i in range(5):
    for j in range(pd_csv.shape[1]):
        if ('TypeofInvestment' in str(pd_csv.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            type_of_investment_index = j
            break
        elif ('Company' in str(pd_csv.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            company_first_index = j
print("company_first_index",company_first_index)



Subtotal [188, 257, 277]
Total list [275]
company_first_index 1


In [580]:
pd_csv[:60]

,Levels,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,Levels,Portfolio Company,NaN,Industry,NaN,Type of Investment (1)(2),NaN,Principal Amount,NaN,Cost,NaN,Fair Value (3),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NonControl NonAffiliate Investments,Non–Control / Non–Affiliate Investments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",NaN,Custom Fragrance Producers,NaN,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",NaN,NaN,12506771,NaN,NaN,NaN,12256771.0,NaN,NaN,NaN,12256771.0,NaN,NaN
3,NonControl NonAffiliate Investments,None,None,None,"Common Units (1,250 units)",NaN,NaN,NaN,1250000,NaN,NaN,1250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NonControl NonAffiliate Investments,NaN,NaN,12506771,NaN,NaN,13506771,NaN,NaN,13506771.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,NaN,Specialty Trade Contractors,NaN,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",NaN,4047120,NaN,NaN,4038351.0,NaN,NaN,4038351.0,NaN,NaN,NaN,NaN,NaN
6,NonControl NonAffiliate Investments,None,None,None,"Subordinated Note-PHM (12% Cash, Due 09/12)",NaN,12857,NaN,NaN,12857.0,NaN,NaN,12857,NaN,NaN,NaN,NaN,NaN,NaN
7,NonControl NonAffiliate Investments,None,None,None,"Common Stock-PHM (128,571 shares)",NaN,NaN,NaN,128571,NaN,NaN,128571,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NonControl NonAffiliate Investments,None,None,None,Common Stock Warrants-AA (455 shares),NaN,NaN,NaN,142361,NaN,NaN,242000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NonControl NonAffiliate Investments,NaN,NaN,4059977,NaN,NaN,4322140,NaN,NaN,4421779.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [581]:
print(company_first_index)
# Adding company for the middle values where it might be missed out
for i in range(2, pd_csv.shape[0]):
    for j in range(company_first_index+2):
        row_alphabet_check = False
        if (pd.isnull(pd_csv.iat[i, j]) or (pd_csv.iat[i, j] == 'nan')):
            # print("Index",i,j)
            if (i > 0):
                #print(pd_csv.iat[i-1,j])
                if (pd.isnull(pd_csv.iat[i - 1, j])):
                    1
                else:
                    if (re.search('[a-zA-Z]', str(pd_csv.iat[i - 1, j]))):
                        print(pd_csv.iat[i-1,j])
                        pd_csv.iat[i, j] = pd_csv.iat[i - 1, j]

1
Agilex Flavors & Fragrances, Inc. (3%)*
Agilex Flavors & Fragrances, Inc. (3%)*
Ambient Air Corporation (“AA”) and Peaden-Hobbs Mechanical, LLC (“PHM”) (1%)*
Ambient Air Corporation (“AA”) and Peaden-Hobbs Mechanical, LLC (“PHM”) (1%)*
Ambient Air Corporation (“AA”) and Peaden-Hobbs Mechanical, LLC (“PHM”) (1%)*
Ambient Air Corporation (“AA”) and Peaden-Hobbs Mechanical, LLC (“PHM”) (1%)*
Aramsco, Inc. (0%)*
Assurance Operations Corporation (0%)*
Audio and Video Labs Holdings, Inc. (3)%)*
Audio and Video Labs Holdings, Inc. (3)%)*
Botanical Laboratories, Inc. (0%)*
Capital Contractors, Inc. (2%)*
Capital Contractors, Inc. (2%)*
Carolina Beverage Group, LLC (1%)*
Carolina Beverage Group, LLC (1%)*
Chromaflo Technologies, LLC (5%)*
Chromaflo Technologies, LLC (5%)*
Comverge, Inc. (4%)*
Comverge, Inc. (4%)*
Comverge, Inc. (4%)*
Continental Anesthesia Management, LLC (2%)*
Continental Anesthesia Management, LLC (2%)*
CRS Reprocessing, LLC (7%)*
CRS Reprocessing, LLC (7%)*
CRS Reprocessin

In [582]:
pd_csv[:30]

,Levels,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,Levels,Portfolio Company,NaN,Industry,NaN,Type of Investment (1)(2),NaN,Principal Amount,NaN,Cost,NaN,Fair Value (3),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NonControl NonAffiliate Investments,Non–Control / Non–Affiliate Investments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",NaN,Custom Fragrance Producers,NaN,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",NaN,NaN,12506771,NaN,NaN,NaN,12256771.0,NaN,NaN,NaN,12256771.0,NaN,NaN
3,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",None,None,"Common Units (1,250 units)",NaN,NaN,NaN,1250000,NaN,NaN,1250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",NaN,12506771,NaN,NaN,13506771,NaN,NaN,13506771.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,NaN,Specialty Trade Contractors,NaN,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",NaN,4047120,NaN,NaN,4038351.0,NaN,NaN,4038351.0,NaN,NaN,NaN,NaN,NaN
6,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,None,"Subordinated Note-PHM (12% Cash, Due 09/12)",NaN,12857,NaN,NaN,12857.0,NaN,NaN,12857,NaN,NaN,NaN,NaN,NaN,NaN
7,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,None,"Common Stock-PHM (128,571 shares)",NaN,NaN,NaN,128571,NaN,NaN,128571,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,None,Common Stock Warrants-AA (455 shares),NaN,NaN,NaN,142361,NaN,NaN,242000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,NaN,4059977,NaN,NaN,4322140,NaN,NaN,4421779.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [583]:
for i in range(1):
    for j in range(pd_csv.shape[1]):
        #print(pd_csv.iat[i,j])
        if ('Company' in str(pd_csv.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            company_index = j
        if ('Industry' in str(pd_csv.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            industry_index = j
        elif ('Investment' in str(pd_csv.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            investment_index = j
print("Industry index",industry_index)
print("Investment index",investment_index)
print("Company index",company_index)

industry_col = pd_csv[pd_csv.columns[industry_index]]
investment_col = pd_csv[pd_csv.columns[investment_index]]
investment_col_next = pd_csv[pd_csv.columns[investment_index+1]]
company_col = pd_csv[pd_csv.columns[company_index]]

investment_numeric_mask = pd.notna(investment_col) & (investment_col.str.isnumeric() | investment_col.str.match('^[-]+$')) 
investment_numeric_indexes = investment_col.loc[investment_numeric_mask].index.to_list()
industry_numeric_mask = pd.notna(industry_col) & (industry_col.str.isnumeric() | industry_col.str.match('^[-]+$') | 
                                                 industry_col.str.contains(r'\b\d{1,2}/\d{1,2}/\d{4}\b')   )  
industry_numeric_indexes = industry_col.loc[industry_numeric_mask].index.to_list()

investment_in_company_list = list(industry_numeric_indexes)+list(investment_numeric_indexes)

money_market_index = list()
for comp_index in investment_in_company_list:
    if 'Money Market Funds' in str(pd_csv.loc[comp_index,pd_csv.columns[company_index]]):
        print(pd_csv.loc[comp_index,pd_csv.columns[company_index]])
        money_market_index.append(comp_index)
        
investment_in_company_list = [x for x in investment_in_company_list if x not in money_market_index]
pd_csv.loc[investment_in_company_list,pd_csv.columns[company_index]:] = pd_csv.loc[investment_in_company_list,pd_csv.columns[company_index]:].shift(2,axis=1)


Industry index 3
Investment index 5
Company index 1


In [584]:
pd_csv.loc[industry_numeric_mask]

,Levels,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
4,NonControl NonAffiliate Investments,None,None,"Agilex Flavors & Fragrances, Inc. (3%)*",NaN,12506771,NaN,NaN,13506771,NaN,NaN,13506771.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NonControl NonAffiliate Investments,None,None,Ambient Air Corporation (“AA”) and Peaden-Hobb...,NaN,4059977,NaN,NaN,4322140,NaN,NaN,4421779.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NonControl NonAffiliate Investments,None,None,"Aramsco, Inc. (0%)*",NaN,1231819,NaN,NaN,1156696,NaN,NaN,1156696.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NonControl NonAffiliate Investments,None,None,"Audio and Video Labs Holdings, Inc. (3)%)*",NaN,13002889,NaN,NaN,14042889,NaN,NaN,14042889.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NonControl NonAffiliate Investments,None,None,"Botanical Laboratories, Inc. (0%)*",NaN,-,NaN,NaN,474600,NaN,NaN,1031000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NonControl NonAffiliate Investments,None,None,"Chromaflo Technologies, LLC (5%)*",NaN,16466899,NaN,NaN,18431003,NaN,NaN,19053003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,NonControl NonAffiliate Investments,None,None,"Comverge, Inc. (4%)*",NaN,15042500,NaN,NaN,15751744,NaN,NaN,15751744.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,NonControl NonAffiliate Investments,None,None,"CRS Reprocessing, LLC (7%)*",NaN,24746596,NaN,NaN,25433598,NaN,NaN,27569626.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,NonControl NonAffiliate Investments,None,None,DataSource Incorporated (2%)*,NaN,8584263,NaN,NaN,9425564,NaN,NaN,9425564.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NonControl NonAffiliate Investments,None,None,DCWV Acquisition Corporation (1%)*,NaN,6054683,NaN,NaN,5939149,NaN,NaN,5939149.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [585]:
pd_csv.head(10)

,Levels,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,Levels,Portfolio Company,NaN,Industry,NaN,Type of Investment (1)(2),NaN,Principal Amount,NaN,Cost,NaN,Fair Value (3),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NonControl NonAffiliate Investments,Non–Control / Non–Affiliate Investments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",NaN,Custom Fragrance Producers,NaN,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",NaN,NaN,12506771,NaN,NaN,NaN,12256771.0,NaN,NaN,NaN,12256771.0,NaN,NaN
3,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",None,None,"Common Units (1,250 units)",NaN,NaN,NaN,1250000,NaN,NaN,1250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NonControl NonAffiliate Investments,None,None,"Agilex Flavors & Fragrances, Inc. (3%)*",NaN,12506771,NaN,NaN,13506771,NaN,NaN,13506771.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,NaN,Specialty Trade Contractors,NaN,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",NaN,4047120,NaN,NaN,4038351.0,NaN,NaN,4038351.0,NaN,NaN,NaN,NaN,NaN
6,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,None,"Subordinated Note-PHM (12% Cash, Due 09/12)",NaN,12857,NaN,NaN,12857.0,NaN,NaN,12857,NaN,NaN,NaN,NaN,NaN,NaN
7,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,None,"Common Stock-PHM (128,571 shares)",NaN,NaN,NaN,128571,NaN,NaN,128571,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,None,Common Stock Warrants-AA (455 shares),NaN,NaN,NaN,142361,NaN,NaN,242000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NonControl NonAffiliate Investments,None,None,Ambient Air Corporation (“AA”) and Peaden-Hobb...,NaN,4059977,NaN,NaN,4322140,NaN,NaN,4421779.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [586]:
pd_csv.head(40)

,Levels,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,Levels,Portfolio Company,NaN,Industry,NaN,Type of Investment (1)(2),NaN,Principal Amount,NaN,Cost,NaN,Fair Value (3),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NonControl NonAffiliate Investments,Non–Control / Non–Affiliate Investments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",NaN,Custom Fragrance Producers,NaN,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",NaN,NaN,12506771,NaN,NaN,NaN,12256771.0,NaN,NaN,NaN,12256771.0,NaN,NaN
3,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",None,None,"Common Units (1,250 units)",NaN,NaN,NaN,1250000,NaN,NaN,1250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NonControl NonAffiliate Investments,None,None,"Agilex Flavors & Fragrances, Inc. (3%)*",NaN,12506771,NaN,NaN,13506771,NaN,NaN,13506771.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,NaN,Specialty Trade Contractors,NaN,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",NaN,4047120,NaN,NaN,4038351.0,NaN,NaN,4038351.0,NaN,NaN,NaN,NaN,NaN
6,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,None,"Subordinated Note-PHM (12% Cash, Due 09/12)",NaN,12857,NaN,NaN,12857.0,NaN,NaN,12857,NaN,NaN,NaN,NaN,NaN,NaN
7,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,None,"Common Stock-PHM (128,571 shares)",NaN,NaN,NaN,128571,NaN,NaN,128571,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,None,Common Stock Warrants-AA (455 shares),NaN,NaN,NaN,142361,NaN,NaN,242000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NonControl NonAffiliate Investments,None,None,Ambient Air Corporation (“AA”) and Peaden-Hobb...,NaN,4059977,NaN,NaN,4322140,NaN,NaN,4421779.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [587]:
for i in range(5):
    for j in range(pd_csv.shape[1]):
        if ('Industry' in str(pd_csv.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            principal_index = j
            break
principal_index = 1
print("Principal Index",principal_index)

Principal Index 1


In [588]:
# Identifying index of maturity
i = 0
# Extracting the column data after shares/warrants/principal
try:
    pd_csv_principal = pd_csv.iloc[:, principal_index:]
    pd_csv_principal = pd_csv_principal.replace("%",np.nan)
    pd_csv_principal = pd_csv_principal.replace("\u200b",np.nan)
    pd_csv_principal = pd_csv_principal.replace("\u200b.1",np.nan)
    
    pd_csv_principal = pd.read_csv(StringIO(re.sub(',+', ',', pd_csv_principal.to_csv())))

except:
    print("no Maturity")

In [589]:
pd_csv_principal[120:180]

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
120,120,"Performance Health & Wellness Holdings, Inc. (...",Designer and Manufacturer of Rehabilitation an...,"Subordinated Note (12% Cash, 1% PIK, Due 04/19)",13029262,12839320.0,12839320.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,121,"Performance Health & Wellness Holdings, Inc. (...","Class A Limited Partnership Units (15,000 units)",1500000.0,1500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,122,"Performance Health & Wellness Holdings, Inc. (...",13029262.0,14339320,14339320.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,123,"PowerDirect Marketing, LLC (2%)*",Marketing Services,"Subordinated Note (12% Cash, 2% PIK, Due 05/16)",7660631,7069381.0,7069381.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,124,"PowerDirect Marketing, LLC (2%)*",Common Unit Purchase Warrants,590200,1820000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,125,"PowerDirect Marketing, LLC (2%)*",7660631,7659581.0,8889381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,126,ROM Acquisition Corporation (3%)*,Military and Industrial Vehicles Equipment Man...,"Subordinated Note (12% Cash, 3% PIK, Due 3/17)",13157755,13037948.0,13037948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,127,ROM Acquisition Corporation (3%)*,13157755,13037948.0,13037948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,128,"Sheplers, Inc. (3%)*",Western Apparel Retailer,"Subordinated Note (13.15% Cash, Due 12/16)",8750000,8564503.0,8564503.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,129,"Sheplers, Inc. (3%)*","Subordinated Note (10% Cash, 7% PIK, Due 12/17)",4027230,3964422.0,3964422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [590]:
pd_csv_principal = pd_csv_principal.iloc[:,1:]
# Dropping columns with Nulls
perc = 95  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv_principal.shape[0] + 1)
pd_csv_principal = pd_csv_principal.dropna(axis=1,
                                           thresh=min_count)

In [591]:
pd_csv_principal

,0,1,2,3,4,5
0,Portfolio Company,Industry,Type of Investment (1)(2),Principal Amount,Cost,Fair Value (3)
1,Non–Control / Non–Affiliate Investments:,NaN,NaN,NaN,NaN,NaN
2,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",12506771,12256771.0,12256771.0
3,"Agilex Flavors & Fragrances, Inc. (3%)*","Common Units (1,250 units)",1250000,1250000,NaN,NaN
4,"Agilex Flavors & Fragrances, Inc. (3%)*",12506771,13506771,13506771.0,NaN,NaN
...,...,...,...,...,...,...
270,Gerli & Company (0%)*,"Class A Preferred Shares (1,211 shares)",855000,-,NaN,NaN
271,Gerli & Company (0%)*,Class C Preferred Shares (744 shares),-,-,NaN,NaN
272,Gerli & Company (0%)*,Class E Preferred Shares (400 shares),161440,-,NaN,NaN
273,Gerli & Company (0%)*,Common Stock (300 shares),100000,-,NaN,NaN


In [592]:
#Changes

for i in range(1):
    for j in range(pd_csv_principal.shape[1]):
        #print(i,j)
        #print(str(pd_csv_principal.iat[i, j]).translate({ord(c): None for c in string.whitespace}))
        try:
            if ('NetAssets' in str(pd_csv_principal.iat[i, j]).translate({ord(c): None for c in string.whitespace})):
                net_assets_index = j
                print("Net Assets Index", net_assets_index)
        except:
            print("Net Assets not present")
        if ('FairValue' in str(pd_csv_principal.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            fair_index = j
        if ('AcquisitionDate' in str(pd_csv_principal.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            acq_date_index = j
        if ('Investment' in str(pd_csv_principal.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            invest_index = j
        if ('Principal' in str(pd_csv_principal.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            prin_value_index = j
        if ('Maturity' in str(pd_csv_principal.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            maturity_index = j
            
print("Principal Index", principal_index)
print("Investment Index", invest_index)
print("Principal Value Index", prin_value_index)
try:
    print("Maturity Index", maturity_index)
except:
    print("Maturity column missing")
try:
    print("Acquisition Date Index",acq_date_index)
except:
    print("No Acq date")

Principal Index 1
Investment Index 2
Principal Value Index 3
Maturity column missing
No Acq date


In [593]:
try:
    pd_csv_principal = pd_csv_principal.iloc[:, :net_assets_index + 1]
except:
    pd_csv_principal = pd_csv_principal.iloc[:,:fair_index+1]

In [594]:
for i in range(int(pd_csv_principal.shape[0])):
    for j in range(pd_csv_principal.shape[1]):
        try:
            pd_csv_principal.iat[i,j] = re.sub(r"\(\d+\)", "", pd_csv_principal.iat[i,j])
            if(pd_csv_principal.iat[i,j] == ''):
                pd_csv_principal.iat[i,j] = np.nan
        except:
            print("Bracket digits not removed")

Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
B

In [595]:
pd_csv_principal[:20]

,0,1,2,3,4,5
0,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value
1,Non–Control / Non–Affiliate Investments:,NaN,NaN,NaN,NaN,NaN
2,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",12506771,12256771.0,12256771.0
3,"Agilex Flavors & Fragrances, Inc. (3%)*","Common Units (1,250 units)",1250000,1250000,NaN,NaN
4,"Agilex Flavors & Fragrances, Inc. (3%)*",12506771,13506771,13506771.0,NaN,NaN
5,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",4047120,4038351.0,4038351.0
6,Ambient Air Corporation (“AA”) and Peaden-Hobb...,"Subordinated Note-PHM (12% Cash, Due 09/12)",12857,12857.0,12857,NaN
7,Ambient Air Corporation (“AA”) and Peaden-Hobb...,"Common Stock-PHM (128,571 shares)",128571,128571,NaN,NaN
8,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Common Stock Warrants-AA (455 shares),142361,242000,NaN,NaN
9,Ambient Air Corporation (“AA”) and Peaden-Hobb...,4059977,4322140,4421779.0,NaN,NaN


In [596]:
#CHANGES

principal_cols = pd_csv_principal.columns[principal_index]
principal_value_cols = pd_csv_principal.columns[prin_value_index]

try:
    acq_date_cols = pd_csv_principal.columns[acq_date_index]
except:
    acq_date_cols = list()
    print("No Acq Date")
try:
    maturity_cols = pd_csv_principal.columns[maturity_index]
except:
    maturity_cols = list()
    print("No Maturity Column")

if(len(maturity_cols)>0 and len(acq_date_cols)>0):
    last_1_nan_list = pd_csv_principal.index[(pd_csv_principal.iloc[:, -1].isnull()) & (pd_csv_principal.iloc[:, -2].notnull()) & (
        pd_csv_principal.iloc[:, -3].notnull())].to_list()
    pd_csv_principal.loc[last_1_nan_list,principal_cols:] = pd_csv_principal.loc[last_1_nan_list,principal_cols:].shift(1,axis=1)
elif(len(maturity_cols)==0 and len(acq_date_cols)>0):
    last_1_nan_list_date_in_invest = pd_csv_principal.index[(pd_csv_principal.iloc[:, -1].isnull()) & (pd_csv_principal.iloc[:, -2].notnull()) & 
                    (pd_csv_principal.iloc[:, -3].notnull()) & pd_csv_principal.iloc[:,invest_index].str.contains('/')].to_list()
    pd_csv_principal.loc[last_1_nan_list_date_in_invest,principal_cols:] = pd_csv_principal.loc[last_1_nan_list_date_in_invest,principal_cols:].shift(1,axis=1)
    
    last_1_nan_list = pd_csv_principal.index[(pd_csv_principal.iloc[:, -1].isnull()) & (pd_csv_principal.iloc[:, -2].notnull()) & 
                    (pd_csv_principal.iloc[:, -3].notnull())].to_list()
    pd_csv_principal.loc[last_1_nan_list,principal_value_cols:] = pd_csv_principal.loc[last_1_nan_list,principal_value_cols:].shift(1,axis=1)
#Need to shift if last is Nan and acquisition date has date then do above
#Otherwise need to shift only the Principal value to the right
else:
    last_1_nan_list = pd_csv_principal.index[(pd_csv_principal.iloc[:, -1].isnull()) & (pd_csv_principal.iloc[:, -2].notnull()) & (
        pd_csv_principal.iloc[:, -3].notnull())].to_list()
    pd_csv_principal.loc[last_1_nan_list,principal_cols:] = pd_csv_principal.loc[last_1_nan_list,principal_cols:].shift(1,axis=1)

No Acq Date
No Maturity Column


In [597]:
pd_csv_principal.loc[last_1_nan_list]

,0,1,2,3,4,5
6,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,"Subordinated Note-PHM (12% Cash, Due 09/12)",12857,12857.0,12857
12,Assurance Operations Corporation (0%)*,None,Metal Fabrication,Common Stock (517 shares),516867,905000.0
17,"Botanical Laboratories, Inc. (0%)*",None,Nutritional Supplement Manufacturing and Distr...,"Common Unit Warrants (998,680 units)",474600,1031000.0
22,"Carolina Beverage Group, LLC (1%)*",None,Beverage Manufacturing and Packaging,"Class A Units (11,974 units)",1077615,1367000.0
36,"CRS Reprocessing, LLC (7%)*",None,"Subordinated Note (10% Cash, 4% PIK, Due 11/15)",12978264,11746721.0,11746721
47,"DLR Restaurants, LLC (3%)*",None,"Subordinated Note (12% Cash, 4% PIK, Due 03/16)",783243,768243.0,768243
64,Garden Fresh Restaurant Corp. (0%)*,None,Restaurant,"Membership Units (5,000 units)",500000,295000.0
71,"Hatch Chile Co., LLC (1%)*",None,"Subordinated Note (14% Cash, Due 07/15)",800000,697713.0,697713
75,"Home Physicians, LLC (“HP”) and Home Physician...",None,"Subordinated Note-HPH (4% Cash, 6% PIK, Due 03...",1364331,1303361.0,-
76,"Home Physicians, LLC (“HP”) and Home Physician...",None,"Senior Subordinated Note-HP (14% Cash, 2% PIK,...",612245,602472.0,602472


In [598]:
principal_cols

'1'

In [599]:
pd_csv_principal[:60]

,0,1,2,3,4,5
0,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value
1,Non–Control / Non–Affiliate Investments:,NaN,NaN,NaN,NaN,NaN
2,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",12506771,12256771.0,12256771.0
3,"Agilex Flavors & Fragrances, Inc. (3%)*","Common Units (1,250 units)",1250000,1250000,NaN,NaN
4,"Agilex Flavors & Fragrances, Inc. (3%)*",12506771,13506771,13506771.0,NaN,NaN
5,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",4047120,4038351.0,4038351.0
6,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,"Subordinated Note-PHM (12% Cash, Due 09/12)",12857,12857.0,12857
7,Ambient Air Corporation (“AA”) and Peaden-Hobb...,"Common Stock-PHM (128,571 shares)",128571,128571,NaN,NaN
8,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Common Stock Warrants-AA (455 shares),142361,242000,NaN,NaN
9,Ambient Air Corporation (“AA”) and Peaden-Hobb...,4059977,4322140,4421779.0,NaN,NaN


In [600]:
pd_csv = pd_csv.iloc[:, :principal_index].reset_index(drop=True)

In [601]:
#pd_csv

In [602]:
# Concatting everything together
pd_csv = pd.concat([pd_csv, pd_csv_principal], axis=1) 



In [603]:
pd_csv.iloc[140:160]

,Levels,0,1,2,3,4,5
140,NonControl NonAffiliate Investments,"Stella Environmental Services, LLC (1%)*",Waste Transfer Stations,"Subordinated Notes (12% Cash, 3% PIK, Due 2/17)",5916326,5788004.0,5788004.0
141,NonControl NonAffiliate Investments,"Stella Environmental Services, LLC (1%)*",Common Stock Purchase Warrants,20000,259000,NaN,NaN
142,NonControl NonAffiliate Investments,"Stella Environmental Services, LLC (1%)*",5916326.0,5808004,6047004.0,NaN,NaN
143,NonControl NonAffiliate Investments,"Syrgis Holdings, Inc. (0%)*",None,Specialty Chemical Manufacturer,"Class C Units (2,114 units)",111037,201281.0
144,NonControl NonAffiliate Investments,"Syrgis Holdings, Inc. (0%)*",111037,201281,NaN,NaN,NaN
145,NonControl NonAffiliate Investments,The Krystal Company (4%)*,Quick Serve Restaurants,"Subordinated Note (12% Cash, 3% PIK, Due 6/17)",12514671,12296595.0,12296595.0
146,NonControl NonAffiliate Investments,The Krystal Company (4%)*,"Class A Units of Limited Partnership (2,000 un...",2000000,3051000,NaN,NaN
147,NonControl NonAffiliate Investments,The Krystal Company (4%)*,12514671,14296595,15347595.0,NaN,NaN
148,NonControl NonAffiliate Investments,"TMR Automotive Service Supply, LLC (1%)*",Automotive Supplies,"Subordinated Note (12% Cash, 1% PIK, Due 03/16)",4750000,4539123.0,4539123.0
149,NonControl NonAffiliate Investments,"TMR Automotive Service Supply, LLC (1%)*","Unit Purchase Warrant (316,858 units)",195000,384000,NaN,NaN


In [604]:
perc = 100  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv.shape[0] + 1)
# Dropping columns which have 100% NaN
pd_csv = pd_csv.dropna(axis=1,
                       thresh=min_count)

new_header = pd_csv.iloc[0]  # grab the first row for the header
#new_header = new_header.str.replace("Shares/Warrants/Principal","Shares Warrants Principal")
pd_csv = pd_csv[1:]  # take the data less the header row
pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header

C:\Users\jaiso\AppData\Local\Temp\ipykernel_1136\4052734264.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header


In [605]:
pd_csv[0:25]

,Levels,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value
1,NonControl NonAffiliate Investments,Non–Control / Non–Affiliate Investments:,NaN,NaN,NaN,NaN,NaN
2,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",12506771,12256771.0,12256771.0
3,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*","Common Units (1,250 units)",1250000,1250000,NaN,NaN
4,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",12506771,13506771,13506771.0,NaN,NaN
5,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",4047120,4038351.0,4038351.0
6,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,"Subordinated Note-PHM (12% Cash, Due 09/12)",12857,12857.0,12857
7,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,"Common Stock-PHM (128,571 shares)",128571,128571,NaN,NaN
8,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Common Stock Warrants-AA (455 shares),142361,242000,NaN,NaN
9,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,4059977,4322140,4421779.0,NaN,NaN
10,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",Environmental Emergency Products Distributor,"Subordinated Note (12% Cash, 2% PIK, Due 03/14)",1231819,1156696.0,1156696.0


In [606]:
pd_csv.columns

Index(['Levels', 'Portfolio Company', 'Industry', 'Type of Investment  ',
       'Principal  Amount', 'Cost', 'Fair  Value  '],
      dtype='object', name=0)

In [607]:
pd_csv = pd_csv.replace("~", ",", regex=True)
#pd_csv = pd_csv.iloc[:, 1:]
pd_csv.columns.values[0] = "Levels"# Dropping columns which NaN as column names
pd_csv = pd_csv.loc[:, pd_csv.columns.notna()]
maturity_cols = [col for col in pd_csv.columns if 'Maturity' in col]
if(len(maturity_cols)>0):
    pd_csv = pd_csv.dropna(thresh=len(pd_csv.columns) -4) 
else:
    pd_csv = pd_csv.dropna(thresh=len(pd_csv.columns) -2) 


In [608]:
pd_csv.head(60)

,Levels,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value
2,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",12506771,12256771.0,12256771.0
3,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*","Common Units (1,250 units)",1250000,1250000,NaN,NaN
4,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",12506771,13506771,13506771.0,NaN,NaN
5,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",4047120,4038351.0,4038351.0
6,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,"Subordinated Note-PHM (12% Cash, Due 09/12)",12857,12857.0,12857
7,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,"Common Stock-PHM (128,571 shares)",128571,128571,NaN,NaN
8,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Common Stock Warrants-AA (455 shares),142361,242000,NaN,NaN
9,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,4059977,4322140,4421779.0,NaN,NaN
10,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",Environmental Emergency Products Distributor,"Subordinated Note (12% Cash, 2% PIK, Due 03/14)",1231819,1156696.0,1156696.0
11,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",1231819,1156696,1156696.0,NaN,NaN


In [609]:
perc = 95  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv.shape[0] + 1)
pd_csv = pd_csv.dropna(axis=1,thresh=min_count)

In [610]:
pd_csv.isnull().sum()/len(pd_csv)

0
Levels                  0.000000
Portfolio Company       0.000000
Industry                0.112403
Type of Investment      0.000000
Principal  Amount       0.000000
Cost                    0.627907
Fair  Value             0.627907
dtype: float64

In [611]:
try:
    pd_csv.drop('​.1', inplace=True, axis=1)
except:
    print(".1 column not present")

.1 column not present


In [612]:
#Appending to Type of Investment from Company column

type_invest_append = ['(Revolver)','(First Out)','(Last Out)','Series A-1','Series A','Series B-1','Series B','Series C','Series A-1',
                      'Series B-1','Class A','Class B','Class C','Class D','Membership Unit Warrants','Series M-1']
try:
    comp_cols = [col for col in pd_csv.columns if 'Investment ,' in col]
    type_inv_cols = [col for col in pd_csv.columns if 'Type of Investment' in col]

    invest_append_list = list()
    for i in type_invest_append:
        print(i)
        if(pd_csv[comp_cols[0]].str.contains(i).any() == True):
            invest_append_list = pd_csv.index[pd_csv[comp_cols[0]].str.contains(i)].to_list()
            for j in invest_append_list:
                if(i in str(pd_csv.loc[int(j),comp_cols[0]])):
                    pd_csv.loc[int(j),comp_cols[0]] = pd_csv.loc[int(j),comp_cols[0]].replace(i,"")
                    pd_csv.loc[int(j),type_inv_cols[0]] = type_inv_cols[0] + " " + i
except:
    print("Company Name column different")

(Revolver)
Company Name column different


In [613]:
# Replaceing ( with minus
for i in pd_csv.columns[-3:]:
    pd_csv[i] = pd_csv[i].replace('\\(', '-',regex=True)

In [614]:
pd_csv[:60]

,Levels,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value
2,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",12506771,12256771.0,12256771.0
3,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*","Common Units (1,250 units)",1250000,1250000,NaN,NaN
4,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",12506771,13506771,13506771.0,NaN,NaN
5,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",4047120,4038351.0,4038351.0
6,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,"Subordinated Note-PHM (12% Cash, Due 09/12)",12857,12857.0,12857
7,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,"Common Stock-PHM (128,571 shares)",128571,128571,NaN,NaN
8,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Common Stock Warrants-AA (455 shares),142361,242000,NaN,NaN
9,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,4059977,4322140,4421779.0,NaN,NaN
10,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",Environmental Emergency Products Distributor,"Subordinated Note (12% Cash, 2% PIK, Due 03/14)",1231819,1156696.0,1156696.0
11,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",1231819,1156696,1156696.0,NaN,NaN


In [615]:
#Changes

invest_cols = [col for col in pd_csv.columns if 'Investment' in col]
industry_cols = [col for col in pd_csv.columns if 'Industry' in col]
acq_date_cols = [col for col in pd_csv.columns if 'Acquisition' in col]
principal_cols = [col for col in pd_csv.columns if 'Principal' in col]

last_2_as_nan_list = pd_csv.index[(pd_csv.iloc[:,-1].isnull()) & (pd_csv.iloc[:,-2].isnull())  & (pd_csv.iloc[:,-3].notnull())].to_list()
try:
    if(len(maturity_cols)>0 and len(acq_date_cols)>0):
        pd_csv.loc[last_2_as_nan_list, maturity_cols[0]:] = pd_csv.loc[last_2_as_nan_list, maturity_cols[0]:].shift(2, axis=1)
    elif(len(maturity_cols)==0 and len(acq_date_cols)>0):
        pd_csv.loc[last_2_as_nan_list, industry_cols[0]:] = pd_csv.loc[last_2_as_nan_list, industry_cols[0]:].shift(1, axis=1)
        pd_csv.loc[last_2_as_nan_list, principal_cols[0]:] = pd_csv.loc[last_2_as_nan_list, principal_cols[0]:].shift(1, axis=1)
    else:
        pd_csv.loc[last_2_as_nan_list, invest_cols[0]:] = pd_csv.loc[last_2_as_nan_list, invest_cols[0]:].shift(2, axis=1)
except:
    print("Last 2 not present")



In [616]:
pd_csv.loc[last_2_as_nan_list].head(60)

,Levels,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value
3,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*","Common Units (1,250 units)",None,None,1250000,1250000
4,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",12506771,None,None,13506771,13506771.0
7,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,"Common Stock-PHM (128,571 shares)",None,None,128571,128571
8,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Common Stock Warrants-AA (455 shares),None,None,142361,242000
9,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,4059977,None,None,4322140,4421779.0
11,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",1231819,None,None,1156696,1156696.0
15,NonControl NonAffiliate Investments,"Audio and Video Labs Holdings, Inc. %)*",Common Units (134 units),None,None,1300000.0,1300000
16,NonControl NonAffiliate Investments,"Audio and Video Labs Holdings, Inc. %)*",13002889,None,None,14042889,14042889.0
18,NonControl NonAffiliate Investments,"Botanical Laboratories, Inc. (0%)*",-,None,None,474600,1031000.0
20,NonControl NonAffiliate Investments,"Capital Contractors, Inc. (2%)*",Common Stock Warrants (20 shares),None,None,492000,339000


In [617]:
#Changes

last_3_nan_list = pd_csv.index[
            (pd_csv.iloc[:, -1].isnull()) & (pd_csv.iloc[:, -2].isnull()) & (pd_csv.iloc[:, -3].isnull())].to_list()
if(len(maturity_cols)>0):
    industry_cols = [col for col in pd_csv.columns if 'Industry' in col]
    #floor_cols = [col for col in pd_csv.columns if 'Floor' in col]
    pd_csv.loc[last_3_nan_list, industry_cols[0]:] = pd_csv.loc[last_3_nan_list, industry_cols[0]:].shift(1, axis=1)
    #pd_csv.loc[last_3_nan_list, floor_cols[0]:] = pd_csv.loc[last_3_nan_list, floor_cols[0]:].shift(2, axis=1)
else:
    pd_csv.drop(last_3_nan_list,inplace=True)


In [618]:
pd_csv.loc[last_3_nan_list].head(60)

,Levels,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value


In [619]:
industry_cols = [col for col in pd_csv.columns if 'Industry' in col]
investements_nan = pd_csv.index[pd_csv[invest_cols[0]].isnull()].to_list()
try:
    pd_csv.loc[investements_nan,[invest_cols[0],industry_cols[0]]] = pd_csv.loc[investements_nan,[industry_cols[0],invest_cols[0]]].values
except:
    print("No nulls in Industry column")

In [620]:
pd_csv.loc[investements_nan].head(60)

,Levels,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value
3,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",None,"Common Units (1,250 units)",None,1250000,1250000
4,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",None,12506771,None,13506771,13506771.0
7,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,"Common Stock-PHM (128,571 shares)",None,128571,128571
8,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,Common Stock Warrants-AA (455 shares),None,142361,242000
9,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,4059977,None,4322140,4421779.0
11,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",None,1231819,None,1156696,1156696.0
15,NonControl NonAffiliate Investments,"Audio and Video Labs Holdings, Inc. %)*",None,Common Units (134 units),None,1300000.0,1300000
16,NonControl NonAffiliate Investments,"Audio and Video Labs Holdings, Inc. %)*",None,13002889,None,14042889,14042889.0
18,NonControl NonAffiliate Investments,"Botanical Laboratories, Inc. (0%)*",None,-,None,474600,1031000.0
20,NonControl NonAffiliate Investments,"Capital Contractors, Inc. (2%)*",None,Common Stock Warrants (20 shares),None,492000,339000


In [621]:
last_nan = list()
pd_csv = pd_csv.reset_index(drop=True)

try:
    for i in range(int(pd_csv.shape[0])):
        if(pd_csv.iat[i,pd_csv.shape[1]-1] == ' ' or pd_csv.iat[i,pd_csv.shape[1]-1] == '  '):
            last_nan.append(i)
            pd_csv.iat[i,pd_csv.shape[1]-1] = np.nan
    #pd_csv.loc[last_nan, maturity_cols[0]:] = pd_csv.loc[last_nan, maturity_cols[0]:].shift(1, axis=1)
except:
    print("Error while switching for last column")

In [622]:
principal_cols = [col for col in pd_csv.columns if 'Principal' in col]

pd_csv[principal_cols[0]] = pd_csv[principal_cols[0]].fillna("")

#Finding out if the column contains string values, as Principal should only contain numbers.
alpha_in_principal = pd_csv.index[pd_csv[principal_cols[0]].str.contains(r'[a-zA-Z]')].to_list()

#Shifting the values back by one column
pd_csv.loc[alpha_in_principal, industry_cols[0]:principal_cols[0]] = pd_csv.loc[alpha_in_principal, industry_cols[0]:principal_cols[0]].shift(-1, axis=1)

In [623]:
pd_csv[:60]

,Levels,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value
0,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",12506771,12256771.0,12256771.0
1,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",None,"Common Units (1,250 units)",,1250000,1250000
2,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",None,12506771,,13506771,13506771.0
3,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",4047120,4038351.0,4038351.0
4,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,"Subordinated Note-PHM (12% Cash, Due 09/12)",12857,12857.0,12857
5,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,"Common Stock-PHM (128,571 shares)",,128571,128571
6,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,Common Stock Warrants-AA (455 shares),,142361,242000
7,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,None,4059977,,4322140,4421779.0
8,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",Environmental Emergency Products Distributor,"Subordinated Note (12% Cash, 2% PIK, Due 03/14)",1231819,1156696.0,1156696.0
9,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",None,1231819,,1156696,1156696.0


In [624]:
#print(type_of_investment_index)
# Adding company for the middle values where it might be missed out
for i in range(0, pd_csv.shape[0]):
    for j in range(investment_index+1):
        row_alphabet_check = False
        if (pd.isnull(pd_csv.iat[i, j]) or (pd_csv.iat[i, j] == 'nan')):
            # print("Index",i,j)
            if (i > 0):
                # print(pd_csv.iat[i-1,j])
                if (pd.isnull(pd_csv.iat[i - 1, j])):
                    1
                else:
                    if (re.search('[a-zA-Z]', str(pd_csv.iat[i - 1, j]))):
                        for k in range(pd_csv.shape[1]):
                            if (re.search('[a-zA-Z]', str(pd_csv.iat[i, k])) and pd.notnull(pd_csv.iat[i, k])):
                                # print("Alpha Check",pd_csv.iat[i,k])
                                row_alphabet_check = True
                                break
                        if (row_alphabet_check):
                            # print(pd_csv.iat[i-1,j])
                            pd_csv.iat[i, j] = pd_csv.iat[i - 1, j]



In [625]:
pd_csv[:15]

,Levels,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value
0,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",12506771,12256771.0,12256771.0
1,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,"Common Units (1,250 units)",,1250000,1250000
2,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,12506771,,13506771,13506771.0
3,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",4047120,4038351.0,4038351.0
4,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Subordinated Note-PHM (12% Cash, Due 09/12)",12857,12857.0,12857
5,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Common Stock-PHM (128,571 shares)",,128571,128571
6,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,Common Stock Warrants-AA (455 shares),,142361,242000
7,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,4059977,,4322140,4421779.0
8,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",Environmental Emergency Products Distributor,"Subordinated Note (12% Cash, 2% PIK, Due 03/14)",1231819,1156696.0,1156696.0
9,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",Environmental Emergency Products Distributor,1231819,,1156696,1156696.0


In [626]:
 #Finding the Type of Investment column
invest_cols = [col for col in pd_csv.columns if 'Investment' in col]
# Removing rows which contain only numbers in the Type of Investment column
pd_csv = pd_csv[pd.to_numeric(pd_csv[invest_cols[0]], errors="coerce").isnull()]

In [627]:
#Finding out if the column Investment contains values like Level 3 
level_in_investments = pd_csv.index[pd_csv[invest_cols[0]].str.contains('Level')].to_list()
#Dropping it if its there
pd_csv.drop(level_in_investments, inplace=True)

In [628]:
pd_csv = pd_csv.fillna("")


In [629]:
pd_csv.head(60)

,Levels,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value
0,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,"Subordinated Note (12% Cash, 1.5% PIK, Due 06/19)",12506771,12256771.0,12256771.0
1,NonControl NonAffiliate Investments,"Agilex Flavors & Fragrances, Inc. (3%)*",Custom Fragrance Producers,"Common Units (1,250 units)",,1250000,1250000
3,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Subordinated Note-AA (15% Cash, 3% PIK, Due 06...",4047120,4038351.0,4038351.0
4,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Subordinated Note-PHM (12% Cash, Due 09/12)",12857,12857.0,12857
5,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,"Common Stock-PHM (128,571 shares)",,128571,128571
6,NonControl NonAffiliate Investments,Ambient Air Corporation (“AA”) and Peaden-Hobb...,Specialty Trade Contractors,Common Stock Warrants-AA (455 shares),,142361,242000
8,NonControl NonAffiliate Investments,"Aramsco, Inc. (0%)*",Environmental Emergency Products Distributor,"Subordinated Note (12% Cash, 2% PIK, Due 03/14)",1231819,1156696.0,1156696.0
10,NonControl NonAffiliate Investments,Assurance Operations Corporation (0%)*,Metal Fabrication,Common Stock -517 shares),,516867,905000.0
11,NonControl NonAffiliate Investments,"Audio and Video Labs Holdings, Inc. %)*",Manufacturer and Distributor for Independent A...,"Subordinated Note (12% Cash, 2% PIK, Due 06/18)",13002889,12742889.0,12742889.0
12,NonControl NonAffiliate Investments,"Audio and Video Labs Holdings, Inc. %)*",Manufacturer and Distributor for Independent A...,Common Units (134 units),,1300000.0,1300000


In [633]:
pd_csv[40:100]

,Levels,Portfolio Company,Industry,Type of Investment,Principal Amount,Cost,Fair Value
54,NonControl NonAffiliate Investments,"Empire Facilities Management Group, Inc. (1%)*","Retail, Restaurant, and Commercial Facilities ...","Convertible Preferred Units (2,500 units)",,250000.0,250000
56,NonControl NonAffiliate Investments,"Foodstate, Inc. (1%)*",Nutritional Supplement Manufacturing and Distr...,"Subordinated Note (12% Cash, 3.8% PIK, Due 10/16)",5480349,5398773.0,5398773.0
58,NonControl NonAffiliate Investments,"Frozen Specialties, Inc. (1%)*",Frozen Foods Manufacturer,"Subordinated Note (13% Cash, 5% PIK, Due 05/16)",6245877,6190449.0,6190449.0
60,NonControl NonAffiliate Investments,Garden Fresh Restaurant Corp. (0%)*,Restaurant,"Membership Units -5,000 units)",,500000,295000.0
61,NonControl NonAffiliate Investments,Glencoe Business Services Holdings (5%)*,Business Process Outsourcing Provider,"Subordinated Note (12% Cash, 2.5% PIK, Due 06/18)",20001388,19601388.0,19601388.0
63,NonControl NonAffiliate Investments,Grindmaster-Cecilware Corp. (1%)*,Food Services Equipment Manufacturer,"Subordinated Note (12% Cash, 6% PIK, Due 04/16)",6667971,6612541.0,5894000.0
65,NonControl NonAffiliate Investments,"Hatch Chile Co., LLC (1%)*",Food Products Distributor,"Senior Note (19% Cash, Due 07/15)",3600000,3530928.0,3530928.0
66,NonControl NonAffiliate Investments,"Hatch Chile Co., LLC (1%)*",Food Products Distributor,"Subordinated Note (14% Cash, Due 07/15)",800000,697713.0,697713
67,NonControl NonAffiliate Investments,"Hatch Chile Co., LLC (1%)*",Food Products Distributor,"Unit Purchase Warrant (5,265 units)",,149800,229000
69,NonControl NonAffiliate Investments,"Home Physicians, LLC (“HP”) and Home Physician...",In-home Primary Care Physician Services,"Subordinated Note-HP (12% Cash, 5% PIK, Due 03...",11208476,10628482.0,6219000.0
